## Practical project 1 - 01 - Dataset adjustments

Explainable Automated Machine Learning course, LTAT.02.023
University of Tartu, Institute of Computer Science

Each team will work on a machine learning problem from end-to-end.

#### Project's general description:
<details>
  <summary>Click here for more information!</summary>

  Each team will work on a machine learning problem from end-to-end.

  **Step 1:** Choose a dataset. Build and train a baseline for comparison. To construct the baseline you do the following:
  * Try a set of possible machine learning algorithms (**13 algorithms**) using their **default hyperparameters** and choose the one with the highest performance for comparison.


  **Step 2:** Based on the problem at hand, you study:
  * the **potential pipeline structure**,
  * **algorithms**
  * or **feature transformers** at each step,
  * **hyper-parameters ranges**.
  
  Use hyperOpt with the potential search space to beat the baseline.


  **Step 3:** Monitor the performance of the constructed pipeline from the previous step across different time budgets (number of iterations) and report the least time budget that you are able to outperform the baseline.


  **Step 4:** Determine whether the difference in performance between the constructed pipeline and the baseline is statistically significant.

</details>

#### Dataset used:
<details>
  <summary>Click here for more information!</summary>

  Dataset is taken from Kaggle competition - Drinking Water Quality Prediction. The goal of the competition is to create a model that predicts the water quality in Estonian water stations based on the government's open data of the previous measurements.

  [Reference](https://www.kaggle.com/competitions/copy-of-drinking-water-quality)
</details>

#### References:
<details>
  <summary>Click here for more information!</summary>

  [MLJ for Data Scientists in Two Hours](https://juliaai.github.io/DataScienceTutorials.jl/end-to-end/telco/)
</details>

### Activate the current project. Check the packages available

In [ ]:
using Pkg

Pkg.activate(".")
Pkg.status()

### Get packages to use

In [2]:
using DataFrames
using CSV
using MLJ

### Do initial dataset transformations

#### Get cleaned train/test data

In [9]:
df_train = CSV.read(joinpath(@__DIR__, "data/clean/train_clean.csv"), delim=',', DataFrame)
display(first(df_train, 3))
df_test = CSV.read(joinpath(@__DIR__, "data/clean/test_clean.csv"), delim=',', DataFrame)
display(first(df_test, 3))

Row,Column1,station_id,Ammonium_2019,Ammonium_2020,Coli-like-bacteria_2019,Coli-like-bacteria_2020,Colony-count-at-22-C_2019,Colony-count-at-22-C_2020,Color-Pt/Co-scale_2019,Color-Pt/Co-scale_2020,Electrical-conductivity_2019,Electrical-conductivity_2020,Escherichia-coli_2019,Escherichia-coli_2020,Iron_2019,Iron_2020,Manganese_2019,Manganese_2020,Odour-dilution-level_2019,Odour-dilution-level_2020,Taste-dilution-degree_2019,Taste-dilution-degree_2020,Turbidity-NTU_2019,Turbidity-NTU_2020,pH _2019,pH _2020,compliance_2019,compliance_2020,compliance_2021
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,2,205,0.05,0.24,0.0,0.0,0.0,0.0,0.0,0.0,448.0,359.0,0.0,0.0,20.0,21.0,10.0,10.0,1.0,1.0,1.0,1.0,1.0,1.0,7.8,7.7,0,0,0
2,3,1228,0.09,0.05,0.0,0.0,15.0,2.0,4.0,4.0,978.0,920.0,0.0,0.0,21.0,10.0,8.75,8.75,1.0,1.25,1.0,1.25,0.5,0.5,7.44,7.58,0,0,0
3,4,470,0.06,0.05,0.0,2.0,1.0,31.0,3.5,3.5,446.0,579.0,0.0,0.0,84.0,3500.0,7.5,7.5,1.0,1.5,1.0,1.5,0.2,44.0,7.7,8.0,0,1,0


Row,Column1,station_id,Ammonium_2019,Ammonium_2020,Coli-like-bacteria_2019,Coli-like-bacteria_2020,Colony-count-at-22-C_2019,Colony-count-at-22-C_2020,Color-Pt/Co-scale_2019,Color-Pt/Co-scale_2020,Electrical-conductivity_2019,Electrical-conductivity_2020,Escherichia-coli_2019,Escherichia-coli_2020,Iron_2019,Iron_2020,Manganese_2019,Manganese_2020,Odour-dilution-level_2019,Odour-dilution-level_2020,Taste-dilution-degree_2019,Taste-dilution-degree_2020,Turbidity-NTU_2019,Turbidity-NTU_2020,pH _2019,pH _2020,compliance_2019,compliance_2020
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,0,163,0.08,0.08,0.0,0.0,12.0,12.0,11.7,12.1,716.0,671.0,0.0,0.0,109.4,98.0,30.0,30.0,1.0,1.0,1.0,1.0,1.18,1.9,8.17,8.12,0,0
2,1,167,0.08,0.08,0.0,0.0,35.3333,73.5,7.7,9.1,996.0,1010.0,0.0,0.0,540.0,78.5,18.0,17.5,1.0,3.0,1.0,3.0,3.9,1.54,7.83,7.81,1,0
3,2,171,0.065,0.085,0.0,0.0,58.6667,135.0,4.0,2.0,734.0,635.0,0.0,0.0,96.0,140.0,6.0,5.0,2.0,1.0,2.0,1.0,1.4,1.5,7.5,7.6,0,1


#### Declare the function required for dataset's columnnames adjustments

* all columnnames to lowercase
* replace suffix like '_20XX'

In [4]:
# Ref, inspired by: dataframes.juliadata.org/stable/man/querying_frameworks/
function adjustcolnames(df, suffix)
    array = []
    for col in names(df)
        push!(array, replace(lowercase(col), suffix => ""))
    end
        rename!(df, Symbol.(array))
end

adjustcolnames (generic function with 1 method)

#### Do relevant adjustments for 2019/2020 datasets

* apply 'adjustcolnames' for each column
* add new column 'year
* union the 2019/2020 dataframes into one
* exclude 'station_id' and 'year'

In [6]:
# Adjust train

# 2019 - apply 'adjustcolnames' for each column
df_train_2019 = select(df_train, :station_id, Cols(r"_2019"))
# add new column 'year'
df_train_2019[!, "year"] .= 2019
adjustcolnames(df_train_2019, "_2019")

# 2020 - apply 'adjustcolnames' for each column
df_train_2020 = select(df_train, :station_id, Cols(r"_2020"))
# add new column 'year'
df_train_2020[!, "year"] .= 2020
adjustcolnames(df_train_2020, "_2020")

# union the 2019/2020 dataframes into one
df_train_all = vcat(df_train_2019, df_train_2020)

# exclude 'station_id' and 'year'
select!(df_train_all, Not([:year, :station_id]))
first(df_train_all, 3)

Row,ammonium,coli-like-bacteria,colony-count-at-22-c,color-pt/co-scale,electrical-conductivity,escherichia-coli,iron,manganese,odour-dilution-level,taste-dilution-degree,turbidity-ntu,ph,compliance
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.05,0.0,0.0,0.0,448.0,0.0,20.0,10.0,1.0,1.0,1.0,7.8,0
2,0.09,0.0,15.0,4.0,978.0,0.0,21.0,8.75,1.0,1.0,0.5,7.44,0
3,0.06,0.0,1.0,3.5,446.0,0.0,84.0,7.5,1.0,1.0,0.2,7.7,0


In [7]:
# Adjust test

# 2019 - apply 'adjustcolnames' for each column
df_test_2019 = select(df_test, :station_id, Cols(r"_2019"))
# add new column 'year'
df_test_2019[!, "year"] .= 2019
adjustcolnames(df_test_2019, "_2019")

# 2020 - apply 'adjustcolnames' for each column
df_test_2020 = select(df_test, :station_id, Cols(r"_2020"))
# add new column 'year'
df_test_2020[!, "year"] .= 2020
adjustcolnames(df_test_2020, "_2020")

# union the 2019/2020 dataframes into one
df_test_all = vcat(df_test_2019, df_test_2020)

# exclude 'station_id' and 'year'
select!(df_test_all, Not([:year, :station_id]))
first(df_test_all, 3)

Row,ammonium,coli-like-bacteria,colony-count-at-22-c,color-pt/co-scale,electrical-conductivity,escherichia-coli,iron,manganese,odour-dilution-level,taste-dilution-degree,turbidity-ntu,ph,compliance
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.08,0.0,12.0,11.7,716.0,0.0,109.4,30.0,1.0,1.0,1.18,8.17,0
2,0.08,0.0,35.3333,7.7,996.0,0.0,540.0,18.0,1.0,1.0,3.9,7.83,1
3,0.065,0.0,58.6667,4.0,734.0,0.0,96.0,6.0,2.0,2.0,1.4,7.5,0


### Save adjusted train/test datasets

In [ ]:
CSV.write(joinpath(@__DIR__, "data/adjusted/train_adjusted.csv"), delim=';', df_train_all)
CSV.write(joinpath(@__DIR__, "data/adjusted/test_adjusted.csv"), delim=';', df_test_all)